In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

# DOWNLOAD DATA

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
train_profile = ProfileReport(train, title="train profile", explorative=True)
test_profile = ProfileReport(test, title="test profile", explorative=True)

In [4]:
train_profile.to_widgets()

Generate report structure: 100%|██████████| 1/1 [00:02<00:00,  2.35s/it]


In [5]:
test_profile.to_widgets()

Generate report structure: 100%|██████████| 1/1 [00:02<00:00,  2.68s/it]
